This code reads the excel file created with tdms_file_reader.ipynb (and modified manually adding the information about Z thresholds, target frequency and if the tdms file is the google drive spreadsheet) and the results.txt files created with analyze_tdms_encoder.ipynb relative to each tdms file. Then it extends the excel table adding information about the encoder thresholds, encoder frequencies and a score which is related to the grade of similarity between the A/B frequency distribution and the Z frequency distribution. However this score function was just a test and should be improved, just do not pay much attention to it. The last cell creates a .csv/excel table with all the information, that will be the most complete database created with these scripts regarding the tdms files.

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
import os
import lib_bbmdev
import logging
from datetime import datetime
import sys
from pathlib import Path

def score(input, target, w_mean=0.35, w_std=0.65):
    score = w_mean*(1-abs(input[0]-target[0])/input[0]) + w_std*(1-abs(input[1]-target[1])/input[1])
    print(w_mean*(1-abs(input[0]-target[0])/input[0]), w_std*(1-abs(input[1]-target[1])/input[1]))
    return score


%matplotlib inline

In [7]:
#initialize
os.chdir(r'C:\Users\ggora\DevelopingToolkit\bbm_run2_tdms')
path = "Data/2023_run2/"
excel_file = "tdms_dataframe_xlsx.xlsx"
df_excel = pd.read_excel(path+excel_file, header=1)
#df_stable = df_excel.dropna()
files = df_excel['File']
#df_stable.set_index("File", inplace = True)
#df_stable
df_excel

,0,File,Iu,Iv,Iw,A,B,Z,H,SV1,...,Count,Vu,Vv,Vw,Duration [s],Sampling Frequency [Hz],Low TH,High TH,Target freq [Hz],Is in Spreadsheet?
0,1,tdms100_room-to-cryo.tdms,1,1,1,1,1,1,1,0,...,1,1,1,1,913.000,1000,NaN,NaN,NaN,Y
1,2,tdms101_room-to-cryo.tdms,1,1,1,1,1,1,1,0,...,1,1,1,1,3098.000,1000,NaN,NaN,NaN,N
2,3,tdms102_room-to-cryo.tdms,1,1,1,1,1,1,1,0,...,1,1,1,1,1379.000,1000,NaN,NaN,NaN,N
3,4,tdms103_room-to-cryo.tdms,1,1,1,1,1,1,1,0,...,1,1,1,1,9348.000,1000,-0.5,0.5,1.0,Y
4,5,tdms104_room-to-cryo.tdms,1,1,1,1,1,1,1,0,...,1,1,1,1,15217.000,1000,-0.5,0.5,0.8,Y
5,6,tdms34_room-to-cryo.tdms,1,1,1,1,1,1,1,1,...,0,1,1,1,425.000,1000,NaN,NaN,NaN,N
6,7,tdms35_room-to-cryo.tdms,1,1,1,1,1,1,1,1,...,0,1,1,1,9959.000,1000,-8.0,-0.5,1.0,Y
7,8,tdms46_room-to-cryo.tdms,1,1,1,1,1,1,1,1,...,0,1,1,1,277.000,1000,NaN,NaN,NaN,N
8,9,tdms47_room-to-cryo.tdms,1,1,1,1,1,1,1,1,...,0,1,1,1,429.000,1000,NaN,NaN,NaN,N
9,10,tdms51_room-to-cryo.tdms,1,1,1,1,1,1,1,1,...,0,1,1,1,313.000,1000,NaN,NaN,NaN,N


In [8]:
os.chdir('C:/Users/ggora/DevelopingToolkit/bbm_run2_tdms/'+path+'/encoder_data')
low_THA, high_THA, low_THB, high_THB = [], [], [], [] 
mean_freqA, mean_freqB, mean_freqZ = [], [], [] 
stdA, stdB, stdZ = [], [], []
scoreA, scoreB = [], []
for file in files:
    dir = f'C:/Users/ggora/DevelopingToolkit/bbm_run2_tdms/{path}/encoder_data/{file}_encoder/'
#filename = f'{dir}/results.txt'
    try:
        results = pd.read_csv(f'{dir}/results.txt', sep = ', ', engine='python')
        results[results=='-'] = np.nan
        low_THA.append(results['Low TH'][0])
        low_THB.append(results['Low TH'][1])
        high_THA.append(results['High TH'][0])
        high_THB.append(results['High TH'][1])
        mean_freqA.append(results['Mean Frequency [Hz]'][0])
        mean_freqB.append(results['Mean Frequency [Hz]'][1])
        mean_freqZ.append(results['Mean Frequency [Hz]'][2])
        stdA.append(results['Std. Dev. [Hz]'][0])
        stdB.append(results['Std. Dev. [Hz]'][1])
        stdZ.append(results['Std. Dev. [Hz]'][2])
        if results['Mean Frequency [Hz]'][0] is not np.nan:
            scrA = score([float(results['Mean Frequency [Hz]'][0]), float(results['Std. Dev. [Hz]'][0])], [float(results['Mean Frequency [Hz]'][2]), float(results['Std. Dev. [Hz]'][2])])
            scoreA.append(scrA)
        else:
            scoreA.append(np.nan)
            print(f'Could not compute score for Encoder A for file {file}.')

        if results['Mean Frequency [Hz]'][1] is not np.nan:
            scrB = score([float(results['Mean Frequency [Hz]'][1]), float(results['Std. Dev. [Hz]'][1])], [float(results['Mean Frequency [Hz]'][2]), float(results['Std. Dev. [Hz]'][2])])
            scoreB.append(scrB)
        else:
            scoreB.append(np.nan)
            print(f'Could not compute score for Encoder B for file {file}.')
        
    except:
        print(f'File {dir}/results.txt not found.')
        low_THA.append(np.nan)
        low_THB.append(np.nan)
        high_THA.append(np.nan)
        high_THB.append(np.nan)
        mean_freqA.append(np.nan)
        mean_freqB.append(np.nan)
        mean_freqZ.append(np.nan)
        stdA.append(np.nan)
        stdB.append(np.nan)
        stdZ.append(np.nan)
        scoreA.append(np.nan)
        scoreB.append(np.nan)
#commute eerything to float
low_THA, high_THA, low_THB, high_THB = np.array(low_THA, dtype=np.float64), np.array(high_THA, dtype=np.float64), np.array(low_THB, dtype=np.float64), np.array(high_THB, dtype=np.float64) 
mean_freqA, mean_freqB, mean_freqZ = np.array(mean_freqA, dtype=np.float64), np.array(mean_freqB, dtype=np.float64), np.array(mean_freqZ, dtype=np.float64)
stdA, stdB, stdZ = np.array(stdA, dtype=np.float64), np.array(stdB, dtype=np.float64), np.array(stdZ, dtype=np.float64)
df_excel['A Score'] = scoreA
df_excel['B Score'] = scoreB
df_excel['Z Mean Frequency [Hz]'] = mean_freqZ
df_excel['A Mean Frequency [Hz]'] = mean_freqA
df_excel['B Mean Frequency [Hz]'] = mean_freqB
df_excel['Z Std. Dev. [Hz]'] = stdZ
df_excel['A Std. Dev. [Hz]'] = stdA
df_excel['B Std. Dev. [Hz]'] = stdB
df_excel['A Low TH'] = low_THA
df_excel['A High TH'] = high_THA
df_excel['B Low TH'] = low_THB
df_excel['B High TH'] = high_THB

df_excel = df_excel[[0, 'File', 'Iu', 'Iv', 'Iw', 'A', 'B', 'Z', 'H', 'SV1', 'SV2', 'SV3', 'Count', 'Vu', 'Vv', 'Vw', 'Duration [s]', 'Sampling Frequency [Hz]', 'A Score', 'B Score',  'Target freq [Hz]',  'Z Mean Frequency [Hz]', 'Z Std. Dev. [Hz]', 'A Mean Frequency [Hz]',  'A Std. Dev. [Hz]','B Mean Frequency [Hz]', 'B Std. Dev. [Hz]', 'Low TH', 'High TH','A Low TH', 'A High TH', 'B Low TH', 'B High TH', 'Is in Spreadsheet?']]
df_excel_final = df_excel.rename(columns={'Low TH': 'Z Low TH', 'High TH':'Z High TH'})
df_excel_final

File C:/Users/ggora/DevelopingToolkit/bbm_run2_tdms/Data/2023_run2//encoder_data/tdms100_room-to-cryo.tdms_encoder//results.txt not found.
File C:/Users/ggora/DevelopingToolkit/bbm_run2_tdms/Data/2023_run2//encoder_data/tdms101_room-to-cryo.tdms_encoder//results.txt not found.
File C:/Users/ggora/DevelopingToolkit/bbm_run2_tdms/Data/2023_run2//encoder_data/tdms102_room-to-cryo.tdms_encoder//results.txt not found.
0.3497175130556335 0.19004437644575004
0.3496472432004603 0.1693659752328003
0.34979515467299993 0.09172604227179844
0.3498794213060001 0.06951495809602178
File C:/Users/ggora/DevelopingToolkit/bbm_run2_tdms/Data/2023_run2//encoder_data/tdms34_room-to-cryo.tdms_encoder//results.txt not found.
Could not compute score for Encoder A for file tdms35_room-to-cryo.tdms.
0.34978959995019976 0.23869279614772332
File C:/Users/ggora/DevelopingToolkit/bbm_run2_tdms/Data/2023_run2//encoder_data/tdms46_room-to-cryo.tdms_encoder//results.txt not found.
File C:/Users/ggora/DevelopingToolkit/

,0,File,Iu,Iv,Iw,A,B,Z,H,SV1,...,A Std. Dev. [Hz],B Mean Frequency [Hz],B Std. Dev. [Hz],Z Low TH,Z High TH,A Low TH,A High TH,B Low TH,B High TH,Is in Spreadsheet?
0,1,tdms100_room-to-cryo.tdms,1,1,1,1,1,1,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y
1,2,tdms101_room-to-cryo.tdms,1,1,1,1,1,1,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
2,3,tdms102_room-to-cryo.tdms,1,1,1,1,1,1,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
3,4,tdms103_room-to-cryo.tdms,1,1,1,1,1,1,1,0,...,0.028648,1.006418,0.032145,-0.5,0.5,1.080,1.090,1.080,1.090,Y
4,5,tdms104_room-to-cryo.tdms,1,1,1,1,1,1,1,0,...,0.022177,0.770443,0.029263,-0.5,0.5,1.070,1.082,1.070,1.082,Y
5,6,tdms34_room-to-cryo.tdms,1,1,1,1,1,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
6,7,tdms35_room-to-cryo.tdms,1,1,1,1,1,1,1,1,...,NaN,1.005988,0.024763,-8.0,-0.5,NaN,NaN,1.086,1.092,Y
7,8,tdms46_room-to-cryo.tdms,1,1,1,1,1,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
8,9,tdms47_room-to-cryo.tdms,1,1,1,1,1,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
9,10,tdms51_room-to-cryo.tdms,1,1,1,1,1,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N


In [9]:
df_excel_final.to_csv(r'C:/Users/ggora/DevelopingToolkit/bbm_run2_tdms/Data/2023_run2/tdsm_dataframe_final.csv')
df_excel_final.to_excel(r'C:/Users/ggora/DevelopingToolkit/bbm_run2_tdms/Data/2023_run2/tdsm_dataframe_final.xlsx', engine='xlsxwriter')